# 04 - Comparaison des Modèles v1 vs v2 par Scénario

**Objectif**: Comparer les performances des modèles entraînés sur v1 (best_move uniquement) vs v2 (multi-scénarios) sur différents types de combats.

**Input**: 
- Modèles v1 et v2 dans `models/`
- Datasets de test v2 avec scénarios dans `data/ml/battle_winner_v2/features/`

**Output**: Analyse comparative et recommandations d'utilisation

**Date**: 2026-01-24

---

## Table des Matières

1. [Chargement des Modèles et Données](#1-chargement-des-modèles-et-données)
2. [Évaluation par Scénario](#2-évaluation-par-scénario)
3. [Comparaison Globale](#3-comparaison-globale)
4. [Analyse Détaillée par Scénario](#4-analyse-détaillée-par-scénario)
5. [Analyse des Erreurs](#5-analyse-des-erreurs)
6. [Recommandations](#6-recommandations)
7. [Conclusion](#7-conclusion)

## 1. Chargement des Modèles et Données

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import joblib
import warnings
warnings.filterwarnings('ignore')

# Sklearn
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, classification_report,
    ConfusionMatrixDisplay
)

# Configuration
plt.style.use('default')
sns.set_palette('husl')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.4f}'.format)

# Chemins
BASE_DIR_V1 = Path('../data/ml/battle_winner')
BASE_DIR_V2 = Path('../data/ml/battle_winner_v2')
MODELS_DIR = Path('../models')

print("📦 Bibliothèques chargées")

📦 Bibliothèques chargées


In [2]:
# Vérifier l'existence des modèles
print("=" * 80)
print("🔍 VÉRIFICATION DES MODÈLES DISPONIBLES")
print("=" * 80)

model_v1_path = MODELS_DIR / 'battle_winner_model_v1.pkl'
model_v2_path = MODELS_DIR / 'battle_winner_model_v2.pkl'

if model_v1_path.exists():
    print(f"✅ Modèle v1 trouvé: {model_v1_path}")
    model_v1 = joblib.load(model_v1_path)
    HAS_V1 = True
else:
    print(f"❌ Modèle v1 non trouvé: {model_v1_path}")
    model_v1 = None
    HAS_V1 = False

if model_v2_path.exists():
    print(f"✅ Modèle v2 trouvé: {model_v2_path}")
    model_v2 = joblib.load(model_v2_path)
    HAS_V2 = True
else:
    print(f"❌ Modèle v2 non trouvé: {model_v2_path}")
    model_v2 = None
    HAS_V2 = False

if not HAS_V1 and not HAS_V2:
    print("\n⚠️ Aucun modèle trouvé. Veuillez d'abord entraîner les modèles avec notebook 03.")
elif not HAS_V2:
    print("\n⚠️ Modèle v2 non trouvé. Ce notebook nécessite les deux modèles pour la comparaison.")
else:
    print("\n✅ Modèles chargés avec succès")

🔍 VÉRIFICATION DES MODÈLES DISPONIBLES
❌ Modèle v1 non trouvé: ../models/battle_winner_model_v1.pkl
✅ Modèle v2 trouvé: ../models/battle_winner_model_v2.pkl

✅ Modèles chargés avec succès


In [3]:
# Charger les données de test v2 (avec scénarios)
print("\n" + "=" * 80)
print("📂 CHARGEMENT DES DONNÉES DE TEST V2")
print("=" * 80)

FEATURES_DIR_V2 = BASE_DIR_V2 / 'features'

if FEATURES_DIR_V2.exists():
    X_test_v2 = pd.read_parquet(FEATURES_DIR_V2 / 'X_test.parquet')
    y_test_v2 = pd.read_parquet(FEATURES_DIR_V2 / 'y_test.parquet')['winner']
    
    # Vérifier si scenario_type existe
    if 'scenario_type' in X_test_v2.columns:
        scenario_test = X_test_v2['scenario_type'].copy()
        X_test_v2_ml = X_test_v2.drop(columns=['scenario_type'])
        HAS_SCENARIOS = True
        print(f"✅ Données de test v2 chargées: {X_test_v2_ml.shape}")
        print(f"✅ Scénarios détectés")
        print(f"\n📊 Distribution des scénarios:")
        print(scenario_test.value_counts())
    else:
        X_test_v2_ml = X_test_v2
        scenario_test = None
        HAS_SCENARIOS = False
        print(f"⚠️ Pas de colonne scenario_type dans les données v2")
else:
    print(f"❌ Répertoire des features v2 non trouvé: {FEATURES_DIR_V2}")
    X_test_v2_ml = None
    y_test_v2 = None
    scenario_test = None
    HAS_SCENARIOS = False


📂 CHARGEMENT DES DONNÉES DE TEST V2
✅ Données de test v2 chargées: (179695, 133)
✅ Scénarios détectés

📊 Distribution des scénarios:
scenario_type
all_combinations    137956
random_move          34782
best_move             6957
Name: count, dtype: int64


## 2. Évaluation par Scénario

In [4]:
if HAS_V1 and HAS_V2 and HAS_SCENARIOS:
    print("=" * 80)
    print("🎯 ÉVALUATION DES MODÈLES PAR SCÉNARIO")
    print("=" * 80)
    
    # Prédictions des deux modèles
    y_pred_v1 = model_v1.predict(X_test_v2_ml)
    y_pred_v2 = model_v2.predict(X_test_v2_ml)
    
    y_proba_v1 = model_v1.predict_proba(X_test_v2_ml)[:, 1] if hasattr(model_v1, 'predict_proba') else None
    y_proba_v2 = model_v2.predict_proba(X_test_v2_ml)[:, 1] if hasattr(model_v2, 'predict_proba') else None
    
    # Analyser par scénario
    scenarios = scenario_test.unique()
    comparison_results = []
    
    for scenario in scenarios:
        mask = scenario_test == scenario
        y_true = y_test_v2[mask]
        y_pred_v1_scenario = y_pred_v1[mask]
        y_pred_v2_scenario = y_pred_v2[mask]
        
        # Métriques v1
        acc_v1 = accuracy_score(y_true, y_pred_v1_scenario)
        prec_v1 = precision_score(y_true, y_pred_v1_scenario)
        rec_v1 = recall_score(y_true, y_pred_v1_scenario)
        f1_v1 = f1_score(y_true, y_pred_v1_scenario)
        
        # Métriques v2
        acc_v2 = accuracy_score(y_true, y_pred_v2_scenario)
        prec_v2 = precision_score(y_true, y_pred_v2_scenario)
        rec_v2 = recall_score(y_true, y_pred_v2_scenario)
        f1_v2 = f1_score(y_true, y_pred_v2_scenario)
        
        # ROC-AUC si disponible
        roc_v1 = roc_auc_score(y_true, y_proba_v1[mask]) if y_proba_v1 is not None else None
        roc_v2 = roc_auc_score(y_true, y_proba_v2[mask]) if y_proba_v2 is not None else None
        
        comparison_results.append({
            'Scénario': scenario,
            'Échantillons': len(y_true),
            'Acc v1': acc_v1,
            'Acc v2': acc_v2,
            'Δ Acc': acc_v2 - acc_v1,
            'Δ Acc %': (acc_v2 - acc_v1) / acc_v1 * 100,
            'F1 v1': f1_v1,
            'F1 v2': f1_v2,
            'Δ F1': f1_v2 - f1_v1,
            'ROC-AUC v1': roc_v1,
            'ROC-AUC v2': roc_v2
        })
        
        print(f"\n📊 Scénario: {scenario}")
        print(f"   Échantillons: {len(y_true):,}")
        print(f"   Accuracy v1: {acc_v1:.4f} | v2: {acc_v2:.4f} | Δ: {acc_v2-acc_v1:+.4f} ({(acc_v2-acc_v1)/acc_v1*100:+.2f}%)")
        print(f"   F1-Score v1: {f1_v1:.4f} | v2: {f1_v2:.4f} | Δ: {f1_v2-f1_v1:+.4f}")
        if roc_v1 and roc_v2:
            print(f"   ROC-AUC  v1: {roc_v1:.4f} | v2: {roc_v2:.4f} | Δ: {roc_v2-roc_v1:+.4f}")
    
    comparison_df = pd.DataFrame(comparison_results)
    
else:
    print("⚠️ Impossible d'effectuer la comparaison - modèles ou données manquants")
    comparison_df = None

⚠️ Impossible d'effectuer la comparaison - modèles ou données manquants


## 3. Comparaison Globale

In [5]:
if comparison_df is not None:
    print("\n" + "=" * 80)
    print("📊 TABLEAU RÉCAPITULATIF")
    print("=" * 80)
    print(comparison_df.to_string(index=False))
    
    # Statistiques globales
    print("\n" + "=" * 80)
    print("📈 STATISTIQUES GLOBALES")
    print("=" * 80)
    
    print(f"\n🎯 Amélioration moyenne de l'accuracy: {comparison_df['Δ Acc'].mean():+.4f} ({comparison_df['Δ Acc %'].mean():+.2f}%)")
    print(f"🎯 Amélioration moyenne du F1-Score: {comparison_df['Δ F1'].mean():+.4f}")
    
    print(f"\n📊 Meilleure amélioration: {comparison_df.loc[comparison_df['Δ Acc'].idxmax(), 'Scénario']}")
    print(f"   Δ Accuracy: {comparison_df['Δ Acc'].max():+.4f} ({comparison_df['Δ Acc %'].max():+.2f}%)")
    
    print(f"\n📊 Pire amélioration: {comparison_df.loc[comparison_df['Δ Acc'].idxmin(), 'Scénario']}")
    print(f"   Δ Accuracy: {comparison_df['Δ Acc'].min():+.4f} ({comparison_df['Δ Acc %'].min():+.2f}%)")

In [6]:
if comparison_df is not None:
    # Visualisation de la comparaison
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. Accuracy par scénario
    x_pos = np.arange(len(comparison_df))
    width = 0.35
    
    axes[0, 0].bar(x_pos - width/2, comparison_df['Acc v1'], width, label='v1 (best_move)', color='#3498db', alpha=0.8)
    axes[0, 0].bar(x_pos + width/2, comparison_df['Acc v2'], width, label='v2 (multi-scénarios)', color='#e74c3c', alpha=0.8)
    axes[0, 0].set_xlabel('Scénario', fontsize=11)
    axes[0, 0].set_ylabel('Accuracy', fontsize=11)
    axes[0, 0].set_title('Accuracy par Scénario: v1 vs v2', fontsize=13, fontweight='bold')
    axes[0, 0].set_xticks(x_pos)
    axes[0, 0].set_xticklabels(comparison_df['Scénario'], rotation=45, ha='right')
    axes[0, 0].legend()
    axes[0, 0].grid(axis='y', alpha=0.3)
    axes[0, 0].set_ylim([0.85, 1.0])
    
    # 2. Amélioration de l'accuracy
    colors = ['#2ecc71' if x > 0 else '#e74c3c' for x in comparison_df['Δ Acc']]
    axes[0, 1].barh(comparison_df['Scénario'], comparison_df['Δ Acc'], color=colors, alpha=0.8)
    axes[0, 1].axvline(x=0, color='black', linestyle='--', linewidth=1)
    axes[0, 1].set_xlabel('Δ Accuracy (v2 - v1)', fontsize=11)
    axes[0, 1].set_title('Amélioration de l\'Accuracy par Scénario', fontsize=13, fontweight='bold')
    axes[0, 1].grid(axis='x', alpha=0.3)
    
    # 3. F1-Score par scénario
    axes[1, 0].bar(x_pos - width/2, comparison_df['F1 v1'], width, label='v1', color='#3498db', alpha=0.8)
    axes[1, 0].bar(x_pos + width/2, comparison_df['F1 v2'], width, label='v2', color='#e74c3c', alpha=0.8)
    axes[1, 0].set_xlabel('Scénario', fontsize=11)
    axes[1, 0].set_ylabel('F1-Score', fontsize=11)
    axes[1, 0].set_title('F1-Score par Scénario: v1 vs v2', fontsize=13, fontweight='bold')
    axes[1, 0].set_xticks(x_pos)
    axes[1, 0].set_xticklabels(comparison_df['Scénario'], rotation=45, ha='right')
    axes[1, 0].legend()
    axes[1, 0].grid(axis='y', alpha=0.3)
    axes[1, 0].set_ylim([0.85, 1.0])
    
    # 4. Amélioration en pourcentage
    axes[1, 1].bar(comparison_df['Scénario'], comparison_df['Δ Acc %'], 
                   color=['#2ecc71' if x > 0 else '#e74c3c' for x in comparison_df['Δ Acc %']], alpha=0.8)
    axes[1, 1].axhline(y=0, color='black', linestyle='--', linewidth=1)
    axes[1, 1].set_xlabel('Scénario', fontsize=11)
    axes[1, 1].set_ylabel('Amélioration (%)', fontsize=11)
    axes[1, 1].set_title('Amélioration Relative de l\'Accuracy (%)', fontsize=13, fontweight='bold')
    axes[1, 1].set_xticklabels(comparison_df['Scénario'], rotation=45, ha='right')
    axes[1, 1].grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 4. Analyse Détaillée par Scénario

In [7]:
if HAS_V1 and HAS_V2 and HAS_SCENARIOS:
    print("=" * 80)
    print("🔍 ANALYSE DÉTAILLÉE PAR SCÉNARIO")
    print("=" * 80)
    
    for scenario in scenarios:
        print(f"\n{'='*80}")
        print(f"📊 Scénario: {scenario}")
        print(f"{'='*80}")
        
        mask = scenario_test == scenario
        y_true = y_test_v2[mask]
        y_pred_v1_scenario = y_pred_v1[mask]
        y_pred_v2_scenario = y_pred_v2[mask]
        
        # Classification reports
        print(f"\n📄 Classification Report - Modèle v1:")
        print(classification_report(y_true, y_pred_v1_scenario, 
                                    target_names=['Pokémon A gagne', 'Pokémon B gagne']))
        
        print(f"\n📄 Classification Report - Modèle v2:")
        print(classification_report(y_true, y_pred_v2_scenario, 
                                    target_names=['Pokémon A gagne', 'Pokémon B gagne']))

In [8]:
if HAS_V1 and HAS_V2 and HAS_SCENARIOS and len(scenarios) <= 3:
    # Matrices de confusion côte à côte pour chaque scénario
    for scenario in scenarios:
        mask = scenario_test == scenario
        y_true = y_test_v2[mask]
        y_pred_v1_scenario = y_pred_v1[mask]
        y_pred_v2_scenario = y_pred_v2[mask]
        
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        # v1
        ConfusionMatrixDisplay.from_predictions(
            y_true, y_pred_v1_scenario,
            display_labels=['A gagne', 'B gagne'],
            cmap='Blues', ax=axes[0]
        )
        axes[0].set_title(f'Modèle v1\n{scenario}', fontsize=13, fontweight='bold')
        
        # v2
        ConfusionMatrixDisplay.from_predictions(
            y_true, y_pred_v2_scenario,
            display_labels=['A gagne', 'B gagne'],
            cmap='Reds', ax=axes[1]
        )
        axes[1].set_title(f'Modèle v2\n{scenario}', fontsize=13, fontweight='bold')
        
        plt.suptitle(f'Matrices de Confusion: {scenario}', fontsize=15, fontweight='bold', y=1.02)
        plt.tight_layout()
        plt.show()

## 5. Analyse des Erreurs

In [9]:
if HAS_V1 and HAS_V2 and HAS_SCENARIOS:
    print("=" * 80)
    print("🔍 ANALYSE DES ERREURS")
    print("=" * 80)
    
    # Analyser les cas où v1 se trompe mais v2 est correct, et vice-versa
    v1_correct = (y_pred_v1 == y_test_v2)
    v2_correct = (y_pred_v2 == y_test_v2)
    
    # Cas où v2 améliore (v1 faux, v2 correct)
    v2_fixes = (~v1_correct) & v2_correct
    print(f"\n✅ Cas où v2 corrige les erreurs de v1: {v2_fixes.sum():,} ({v2_fixes.sum()/len(y_test_v2)*100:.2f}%)")
    
    # Cas où v2 régresse (v1 correct, v2 faux)
    v2_breaks = v1_correct & (~v2_correct)
    print(f"❌ Cas où v2 régresse par rapport à v1: {v2_breaks.sum():,} ({v2_breaks.sum()/len(y_test_v2)*100:.2f}%)")
    
    # Les deux se trompent
    both_wrong = (~v1_correct) & (~v2_correct)
    print(f"⚠️ Cas où les deux modèles se trompent: {both_wrong.sum():,} ({both_wrong.sum()/len(y_test_v2)*100:.2f}%)")
    
    # Les deux sont corrects
    both_correct = v1_correct & v2_correct
    print(f"✅ Cas où les deux modèles sont corrects: {both_correct.sum():,} ({both_correct.sum()/len(y_test_v2)*100:.2f}%)")
    
    # Analyse par scénario
    print(f"\n{'='*80}")
    print("📊 ANALYSE PAR SCÉNARIO")
    print(f"{'='*80}")
    
    for scenario in scenarios:
        mask = scenario_test == scenario
        print(f"\n📊 {scenario}:")
        print(f"   v2 corrige v1: {v2_fixes[mask].sum():,} échantillons")
        print(f"   v2 régresse:   {v2_breaks[mask].sum():,} échantillons")
        print(f"   Balance nette: {v2_fixes[mask].sum() - v2_breaks[mask].sum():+,} échantillons")

In [10]:
if HAS_V1 and HAS_V2 and HAS_SCENARIOS:
    # Visualisation de l'analyse des erreurs
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # 1. Distribution globale des corrections/régressions
    categories = ['v2 corrige v1', 'v2 régresse', 'Les deux OK', 'Les deux KO']
    counts = [v2_fixes.sum(), v2_breaks.sum(), both_correct.sum(), both_wrong.sum()]
    colors_cat = ['#2ecc71', '#e74c3c', '#3498db', '#f39c12']
    
    axes[0].pie(counts, labels=categories, autopct='%1.1f%%', colors=colors_cat, startangle=90)
    axes[0].set_title('Distribution des Prédictions\n(v1 vs v2)', fontsize=14, fontweight='bold')
    
    # 2. Balance nette par scénario
    net_improvements = []
    for scenario in scenarios:
        mask = scenario_test == scenario
        net = v2_fixes[mask].sum() - v2_breaks[mask].sum()
        net_improvements.append(net)
    
    colors_net = ['#2ecc71' if x > 0 else '#e74c3c' for x in net_improvements]
    axes[1].bar(scenarios, net_improvements, color=colors_net, alpha=0.8)
    axes[1].axhline(y=0, color='black', linestyle='--', linewidth=1)
    axes[1].set_xlabel('Scénario', fontsize=12)
    axes[1].set_ylabel('Balance Nette\n(corrections - régressions)', fontsize=12)
    axes[1].set_title('Impact Net du Modèle v2 par Scénario', fontsize=14, fontweight='bold')
    axes[1].set_xticklabels(scenarios, rotation=45, ha='right')
    axes[1].grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 6. Recommandations

In [11]:
if comparison_df is not None:
    print("=" * 80)
    print("💡 RECOMMANDATIONS D'UTILISATION")
    print("=" * 80)
    
    # Analyser les résultats pour donner des recommandations
    avg_improvement = comparison_df['Δ Acc %'].mean()
    min_improvement = comparison_df['Δ Acc %'].min()
    max_improvement = comparison_df['Δ Acc %'].max()
    
    print(f"\n📊 Amélioration moyenne: {avg_improvement:+.2f}%")
    print(f"📊 Plage d'amélioration: {min_improvement:+.2f}% à {max_improvement:+.2f}%")
    
    print(f"\n{'='*80}")
    print("🎯 RECOMMANDATIONS")
    print(f"{'='*80}")
    
    if avg_improvement > 1:
        print(f"\n✅ Le modèle v2 apporte une amélioration significative ({avg_improvement:+.2f}% en moyenne)")
        print(f"   ➡️ RECOMMANDATION: Utiliser le modèle v2 en production")
    elif avg_improvement > 0:
        print(f"\n✅ Le modèle v2 apporte une légère amélioration ({avg_improvement:+.2f}% en moyenne)")
        print(f"   ➡️ RECOMMANDATION: Utiliser le modèle v2 pour plus de robustesse")
    else:
        print(f"\n⚠️ Le modèle v2 n'apporte pas d'amélioration ({avg_improvement:+.2f}% en moyenne)")
        print(f"   ➡️ RECOMMANDATION: Conserver le modèle v1 ou revoir la stratégie v2")
    
    print(f"\n{'='*80}")
    print("📝 DÉTAILS PAR SCÉNARIO")
    print(f"{'='*80}")
    
    for _, row in comparison_df.iterrows():
        scenario = row['Scénario']
        delta_pct = row['Δ Acc %']
        
        if delta_pct > 1:
            emoji = "✅"
            message = "Amélioration significative"
        elif delta_pct > 0:
            emoji = "➕"
            message = "Légère amélioration"
        elif delta_pct > -1:
            emoji = "➖"
            message = "Légère régression"
        else:
            emoji = "❌"
            message = "Régression significative"
        
        print(f"\n{emoji} {scenario}:")
        print(f"   {message}: {delta_pct:+.2f}%")
        print(f"   Accuracy: {row['Acc v1']:.4f} → {row['Acc v2']:.4f}")
    
    print(f"\n{'='*80}")
    print("🎯 CONTEXTE D'UTILISATION")
    print(f"{'='*80}")
    
    print(f"\n📌 Modèle v1 (best_move uniquement):")
    print(f"   ✓ Plus simple et rapide")
    print(f"   ✓ Optimisé pour le scénario 'best_move'")
    print(f"   ✗ Peut sous-performer sur d'autres scénarios")
    
    print(f"\n📌 Modèle v2 (multi-scénarios):")
    print(f"   ✓ Plus robuste sur différents types de combats")
    print(f"   ✓ Meilleure généralisation")
    print(f"   ✗ Entraînement plus long (880k échantillons)")
    
    print(f"\n{'='*80}")
    print("🚀 INTÉGRATION API")
    print(f"{'='*80}")
    
    print(f"\n💡 Avec le paramètre available_moves_b de l'API:")
    print(f"   • Si available_moves_b est fourni → Scénario contraint")
    print(f"   • Si available_moves_b est None → Scénario best_move")
    print(f"   • Le modèle v2 est recommandé pour gérer les deux cas")

## 7. Conclusion

### Synthèse de la Comparaison

#### 🎯 Objectifs

Ce notebook a comparé les performances de deux approches de modélisation :
- **Modèle v1** : Entraîné uniquement sur le scénario `best_move` (~34k échantillons)
- **Modèle v2** : Entraîné sur 3 scénarios (`best_move`, `random_move`, `all_combinations`) (~880k échantillons)

#### 📊 Résultats Clés

Consultez les tableaux et graphiques ci-dessus pour :
- L'amélioration de l'accuracy par scénario
- L'analyse des corrections et régressions
- Les matrices de confusion détaillées

#### 💡 Décision Finale

La décision d'utiliser v1 ou v2 dépend de :
1. **Performance globale** : Si v2 améliore significativement la précision
2. **Robustesse** : Capacité à gérer différents types de combats
3. **Coût** : Temps d'entraînement et ressources nécessaires
4. **Usage API** : Besoin de gérer le paramètre `available_moves_b`

#### 🚀 Prochaines Étapes

1. **Déploiement** : Intégrer le modèle choisi dans l'API
2. **Monitoring** : Suivre les performances en production
3. **A/B Testing** : Tester les deux modèles en parallèle si nécessaire
4. **Amélioration continue** : Collecter du feedback et ré-entraîner

---

**✅ Analyse comparative terminée!**